In [1]:
import spiceypy
# Allows us to convert to ephemeris time

In [4]:
import datetime
date_today = datetime.datetime.today()

date_today = date_today.strftime("%Y-%m-%dT00:00:00")

In [5]:
print(date_today)

2024-09-28T00:00:00


In [7]:
spiceypy.furnsh("../kernels/lsk/naif0012.tls.txt")
spiceypy.furnsh("../kernels/spk/de432s.bsp")

In [8]:
et_today_midnight = spiceypy.utc2et(date_today)
print(et_today_midnight)

780753669.1823555


In [11]:
# spkgeo calculates the geometric state (position and velocity) of a target object (the earth) relative to an observer (being the sun)
earth_state_wrt_sun, earth_sun_light_time = spiceypy.spkgeo(targ=399, et=et_today_midnight, 
                                                            ref="ECLIPJ2000", obs=10)
print(earth_state_wrt_sun)

[ 1.49333121e+08  1.31194714e+07 -1.94466328e+03 -3.08353468e+00
  2.95698126e+01 -1.03326126e-03]
